### High-Value Customer Identification

In [8]:
import pandas as pd
import sqlite3
import sql
sql.style = 'plain'
%load_ext sql
connection=sqlite3.connect("question_bank.db")
print("Successfully connected to SQL database")
# Connect to SQLite
%sql sqlite:///question_bank.db
print("success")

The sql extension is already loaded. To reload it, use:
  %reload_ext sql
Successfully connected to SQL database
success


In [10]:
%%sql
DROP TABLE IF EXISTS transactions;
DROP TABLE IF EXISTS fraud_accounts;

CREATE TABLE transactions (
    transaction_id INT PRIMARY KEY,
    sender_id INT,
    receiver_id INT,
    amount DECIMAL(10,2),
    transaction_date DATE
);

CREATE TABLE fraud_accounts (
    account_id INT PRIMARY KEY
);

INSERT INTO transactions (transaction_id, sender_id, receiver_id, amount, transaction_date) VALUES
(1, 1, 2, 1200, DATE('now', '-10 days')),
(2, 2, 3, 6000, DATE('now', '-5 days')),
(3, 3, 4, 800,  DATE('now', '-15 days')),
(4, 4, 5, 2000, DATE('now', '-40 days')),
(5, 5, 6, 7000, DATE('now', '-2 days')),
(6, 6, 7, 500,  DATE('now', '-1 days'));


INSERT INTO fraud_accounts (account_id) VALUES
(3),
(7);


 * sqlite:///question_bank.db
Done.
Done.
Done.
Done.
6 rows affected.
2 rows affected.


[]

In [11]:
%%sql
select * from transactions

 * sqlite:///question_bank.db
Done.


transaction_id,sender_id,receiver_id,amount,transaction_date
1,1,2,1200,2025-08-17
2,2,3,6000,2025-08-22
3,3,4,800,2025-08-12
4,4,5,2000,2025-07-18
5,5,6,7000,2025-08-25
6,6,7,500,2025-08-26


In [12]:
%%sql
select * from fraud_accounts

 * sqlite:///question_bank.db
Done.


account_id
3
7


* Find all unique users who are considered “High-Value Customers”: 
* users who either sent payments greater than 1000 or received payments greater than 5000 within the last month, 
* while excluding fraudulent accounts.

In [37]:
%%sql
-- sender_id -- amount >1000
-- receiver_id -- amount >5000
-- 3 and 7 is fraud

select *,
    case when sender_id then amount>1000 end as sender_amt_greater_1000,
    case when receiver_id then amount>5000 end as receiver_amt_greater_5000
from transactions

 * sqlite:///question_bank.db
Done.


transaction_id,sender_id,receiver_id,amount,transaction_date,sender_amt_greater_1000,receiver_amt_greater_5000
1,1,2,1200,2025-08-17,1,0
2,2,3,6000,2025-08-22,1,1
3,3,4,800,2025-08-12,0,0
4,4,5,2000,2025-07-18,1,0
5,5,6,7000,2025-08-25,1,1
6,6,7,500,2025-08-26,0,0


In [71]:
%%sql

with temp as (
    select *,
        case when sender_id then amount>1000 end as sender_amt_greater_1000,
        case when receiver_id then amount>5000 end as receiver_amt_greater_5000
    from transactions   
)
select sender_id, receiver_id, amount
from temp
where 
    sender_amt_greater_1000=1 or receiver_amt_greater_5000=1 and
    sender_id NOT in (select account_id from fraud_accounts)

 * sqlite:///question_bank.db
Done.


sender_id,receiver_id,amount
1,2,1200
2,3,6000
4,5,2000
5,6,7000


In [61]:
%%sql

with temp as (
    select *,
        case when sender_id then amount>1000 end as sender_amt_greater_1000,
        case when receiver_id then amount>5000 end as receiver_amt_greater_5000
    from transactions   
),
temp2 as (
    select sender_id, receiver_id, amount
    from temp
    where 
        sender_amt_greater_1000=1 or receiver_amt_greater_5000=1  
)
select * from temp2
where receiver_id!=3 and sender_id!=3

 * sqlite:///question_bank.db
Done.


sender_id,receiver_id,amount
1,2,1200
4,5,2000
5,6,7000


* Conditional filtering (amount > 1000 vs amount > 5000)

* Date filtering (last 30 days)

* Excluding from another table (fraud_accounts)

* Combining sender vs receiver (UNION)

### let's try UNION all

In [64]:
%%sql
select * from transactions

 * sqlite:///question_bank.db
Done.


transaction_id,sender_id,receiver_id,amount,transaction_date
1,1,2,1200,2025-08-17
2,2,3,6000,2025-08-22
3,3,4,800,2025-08-12
4,4,5,2000,2025-07-18
5,5,6,7000,2025-08-25
6,6,7,500,2025-08-26


In [66]:
%%sql

select sender_id,amount
from transactions
where 
    amount>1000 and 
    transaction_date >= DATE('now', '-30 days') and
    sender_id not in (select account_id from fraud_accounts)

 * sqlite:///question_bank.db
Done.


sender_id,amount
1,1200
2,6000
5,7000


In [67]:
%%sql

select receiver_id,amount
from transactions
where 
    amount>5000 and
    transaction_date >=date('now','-30 days') and 
    receiver_id not in (select account_id from fraud_accounts)

 * sqlite:///question_bank.db
Done.


receiver_id,amount
6,7000


In [68]:
%%sql

select sender_id,amount
from transactions
where 
    amount>1000 and 
    transaction_date >= DATE('now', '-30 days') and
    sender_id not in (select account_id from fraud_accounts)

union 

select receiver_id,amount
from transactions
where 
    amount>5000 and
    transaction_date >=date('now','-30 days') and 
    receiver_id not in (select account_id from fraud_accounts)

 * sqlite:///question_bank.db
Done.


sender_id,amount
1,1200
2,6000
5,7000
6,7000


In [69]:
%%sql

select distinct sender_id,amount
from transactions
where 
    amount>1000 and 
    transaction_date >= DATE('now', '-30 days') and
    sender_id not in (select account_id from fraud_accounts)
union 

select distinct receiver_id,amount
from transactions
where 
    amount>5000 and
    transaction_date >=date('now','-30 days') and 
    receiver_id not in (select account_id from fraud_accounts)

 * sqlite:///question_bank.db
Done.


sender_id,amount
1,1200
2,6000
5,7000
6,7000


* When you want to evaluate two columns at a time, union is used. 
* Here I wanted to filter out amount for sender_id
* And similarly, filter out amount for receiver_id